# **Soft Labels**

### ¿Qué son los Soft Labels?
Los **soft labels** (etiquetas suaves) son probabilidades asociadas a cada clase frente a etiquetas duras (hard labels). 
- En lugar de asignar una clase específica (por ejemplo, `0` o `1`), las soft labels representan la probabilidad de pertenencia de una muestra a cada clase.

Por ejemplo:
- Hard label: `[0]` (la muestra pertenece a la clase 0).
- Soft label: `[0.8, 0.2]` (la muestra tiene un 80% de probabilidad de pertenecer a la clase 0 y un 20% de pertenecer a la clase 1).

### ¿Por qué usar Soft Labels?
1. **Incorporación de incertidumbre**: Las soft labels permiten modelar la incertidumbre en las etiquetas.
2. **Mejora en el entrenamiento**: Algunos modelos pueden beneficiarse de soft labels para aprender patrones más robustos.
3. **Transferencia de conocimiento**: En aprendizaje por transferencia, las soft labels generadas por un modelo maestro pueden usarse para entrenar un modelo estudiante (técnica conocida como *knowledge distillation*).

## 2. Ejemplo Práctico con Scikit-Learn

En este ejemplo, generaremos datos sintéticos y usaremos soft labels para entrenar un modelo de regresión logística.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
df = pd.read_csv('../../data/titanic/train.csv')
df.columns

df.dropna(inplace=True)

In [ ]:
X = df.drop(['PassengerId', 'Survived', 'Pclass', 'Name',
            'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

y = df['Survived']
X

In [ ]:
lenc = LabelEncoder()
stadscl = StandardScaler()

X['Sex'] = lenc.fit_transform(X['Sex'])
X = stadscl.fit_transform(X)

print('X:', X[:5])

In [ ]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Clasificador clásico

In [ ]:
class_model = LogisticRegression(random_state=42)
class_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
y_pred = class_model.predict(X_test)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

### Crear Soft Labels
Generamos soft labels simulando probabilidades para cada clase. Usaremos un modelo maestro (en este caso, también una regresión logística) para generar estas probabilidades.


In [ ]:
# Entrenar un modelo maestro para generar soft labels
master_model = LogisticRegression(random_state=42)
master_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
soft_labels_train = master_model.predict_proba(X_train)

# Mostrar las primeras 5 soft labels
print("Soft Labels (primeras 5 filas):")
print(soft_labels_train[:5])

### Entrenar un Modelo Estudiante con Soft Labels
Usamos las soft labels generadas para entrenar un modelo lineal

In [ ]:
# Entrenar un modelo con soft labels
reg_model = LinearRegression()
# Usamos la probabilidad de la clase 1
reg_model.fit(X_train, soft_labels_train[:, 1])

# Predecir en el conjunto de prueba
y_pred = reg_model.predict(X_test)
y_pred_proba = (y_pred > 0.5).astype(int)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")

## 3. Resultados y Conclusión

### Resultados
- El modelo estudiante se entrena con soft labels generadas por el modelo maestro.
- Las soft labels permiten al modelo estudiante aprender patrones más suaves y generalizables.

### Conclusión
- Los soft labels son útiles para incorporar incertidumbre en el proceso de entrenamiento.
- Técnicas como *knowledge distillation* utilizan soft labels para transferir conocimiento de un modelo grande a uno más pequeño.